In [218]:
import numpy as np
import cvxpy as cp
from numpy.linalg import matrix_power
import matplotlib.pyplot as plt
import mosek

In [219]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [220]:
def simulation_Euler(Ak, Bk, x, u ):
    '''
    Simulate with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]    
    ''' 
    x_next = Ak @ x + Bk @ u

    
    return x_next

In [221]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

In [222]:
def stack_system(A, B, C, D, E, x_init, N):
    '''
    Stack system matrix for N prediction horizon
    
    x_next = A x_k + B u_k + C w_k
    y_k = D x_k + E w_k
    
    '''

    n = np.shape(A)[1]    #  Dimension of state
    m = np.shape(B)[1]    #  Dimension of input
    d = np.shape(C)[1]    #  Dimension of disturbance
    r = np.shape(D)[0]    #  Dimension of output
    
#     print(Nx,Nu,Nw,Ny)
    Nx = (N+1) * n
    Nu = N * m
    Ny = N * r
    Nw = N * d
    
    Ax = np.zeros([Nx,n])
    Bx = np.zeros([Nx,Nu])
    Cx = np.zeros([Nx,Nw])
    Ay = np.zeros([Ny,n])
    By = np.zeros([Ny,Nu])
    Cy = np.zeros([Ny,Nw])
    Ey = np.zeros([Ny,Nw])
    Cx_tilde = np.zeros([Nx, Nw + 1])
    Cy_tilde = np.zeros([Ny + 1, Nw + 1])
    Ey_tilde = np.zeros([Ny + 1, Nw + 1])
    D_tilde = np.zeros([Ny + 1, Nx])
#     H

# Ax
    for i in range(N+1):
        Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# Bx
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
            mat_temp = A @ mat_temp
# Cx
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
            mat_temp = A @ mat_temp
# Ay
    for i in range(N):
        Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# By
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Cy
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Ey
    for i in range(N):
        Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# Cx_tilde
    Cx_tilde[ : , [0]  ] = Ax @ x_init
    Cx_tilde[ : , 1 : ] = Cx
# Cy_tilde
    Cy_tilde[ r : , [0] ] = Ay @ x_init
    Cy_tilde[ r : , 1 : ] = Cy
# Ey_tilde
    Ey_tilde[0, 0] = 1
    Ey_tilde[1:, 1:] = Ey
# D_tilde
    for i in range(N):
        D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
    return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

In [278]:
def define_loss_func(m, d, r, Nu, Nw, Bx, Cx_tilde, mu, beta, sin_const):
    # Define decision variables for POB affine constrol law
    
    sigma = 1 # var of normal distribution
    
    H_cal_dec = cp.Variable((Nu , 1))
#     print(H_cal_dec)
    # \mathcal{H} matrix
    for i in range(N):
        H_col = cp.Variable((Nu - (i * m) , r))
        if i > 0:
            H_col = cp.vstack([np.zeros([i * m, r]), H_col])
#         print(H_col)
        H_cal_dec = cp.hstack([H_cal_dec,H_col])
#     print(H_cal_dec)
#     print(np.shape(H_cal_dec))
    # Define intermediate decision variables for objective function
    H = cp.Variable((Nu , Nw + 1))
    
    # Define loss function
#     beta = 0.95
    Jx = np.zeros([(N+1)* n, (N+1)* n])
    for i in range(N):
        Jx[i * n : (i+1) * n, i * n : (i+1) * n] = beta**i * Q
    Jx[ N * n: , N * n:] = beta ** N * Qf

    Ju = np.zeros([N * m, N * m])
    for i in range(N):
        Ju[i * m : (i+1) * m, i * m : (i+1) * m] = beta**i * R

    # This is only for var = 1 and mean = 0. Should be modified.
    mu_w = np.vstack([1]+ [mu] * N)
#     M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)
    M_w = np.diag([1] + [sin_const **2 * (1-np.exp(-2*sigma**2))/2 ]* N * d)

    # Intermediate decision variables. Since CVXPY does not support quadratic obj of decision variable matrix.
    H_new_matrix = []
#     for i in range(Nw+1):
#         H_new_matrix += [cp.Variable([Nu,1])]
#     H_new = cp.hstack(H_new_matrix)
    for i in range(Nu):
        H_new_matrix += [cp.Variable((1, Nw+1))]
    H_new = cp.vstack(H_new_matrix)

#     print(H_new.shape)
    # Reformulate the quadratic term
    
    eigval,eigvec = np.linalg.eig(Ju + Bx.T @ Jx @ Bx)
    eigval_mat =  np.diag(eigval)
#     print(Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ np.linalg.inv(eigvec))
    # Loss function
    loss_func = 0

    N_eig = np.shape(eigval)[0]
    I = np.diag([1]*(Nw+1))
    for i in range(N_eig):
        # Reformulate Tr(H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w
#         print(np.shape(H_new_matrix[i].T))
        loss_func += eigval[i] * M_w[i,i] * cp.quad_form(H_new_matrix[i].T, I) # When M_w is identity matrix. Otherwise reformulate system matrix or this line
#     loss_func += cp.trace(2 * Cx_tilde.T @ Jx @ Bx @ eigvec @ H_new  @ M_w)
    loss_func += cp.trace(2 * Cx_tilde.T @ Jx @ Bx @ H @ M_w)
#     loss_func += cp.trace(2 * Cx_tilde.T @ Jx @ Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) @ M_w)
    loss_func += cp.trace(Cx_tilde.T @ Jx @ Cx_tilde @ M_w)
    # Reformulate mu_w.T @ (H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ mu_w
#     loss_func += eigval[0] * cp.quad_form(H_new_matrix[0].T, I) +  2 * mu_w.T @  Cx_tilde.T @ Jx @ Bx @ eigvec @ H_new @ mu_w
    loss_func += eigval[0] * cp.quad_form(H_new_matrix[0].T, I) +  2 * mu_w.T @  Cx_tilde.T @ Jx @ Bx @ H @ mu_w
#     loss_func += eigval[0] * cp.quad_form(H_new_matrix[0].T, I) +  2 * mu_w.T @  Cx_tilde.T @ Jx @ Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) @ mu_w

    loss_func += mu_w.T @ Cx_tilde.T @ Jx @ Cx_tilde @ mu_w
    
    return Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func

In [279]:
def gene_disturbance(N, d, N_sample, sin_const):
    # Generate data: const * sinx

    w_sample = []
    for i in range(N_sample):
        w_temp = sin_const * np.sin(np.random.randn(N*d))
#         w_temp = sin_const * np.zeros([N*d])
#         print(w_temp)
        w_sample += [w_temp]
    W_sample_matrix = np.array(w_sample).T

    W_sample_matrix_ext = np.vstack( [np.ones([1, N_sample]),W_sample_matrix])
    return W_sample_matrix, W_sample_matrix_ext

In [280]:
def define_constraint(W_sample_matrix, W_sample_matrix_ext, eigvec, H_cal_dec, H, H_new, n, Bx, Cx_tilde, Cy_tilde, Ey_tilde, N_sample, sin_const, i_th_state, i_state_ub, epsilon):

    constraint = []
    constraint += [H_new == np.linalg.inv(eigvec) @ H ]
#     constraint += [H_new == eigvec.T @ H ]
    constraint += [H == H_cal_dec @ (Cy_tilde + Ey_tilde) ]
#     constraint += [H_new == np.linalg.inv(eigvec) @ H_cal_dec @ (Cy_tilde + Ey_tilde) ]


#     i_th_state = 1 # 0 for first element, 1 for second element
#     i_state_ub = 0.05

    d_supp = np.vstack( ( sin_const * np.ones([N*d, 1]), sin_const * np.ones([N*d, 1])))
    C_supp = np.vstack( (np.diag([1]*N*d), np.diag([-1]*N*d) ))
#     d_supp = np.vstack( ( 0 * np.ones([N*d, 1]), 0 * np.ones([N*d, 1])))
#     C_supp = np.vstack( (np.diag([0]*N*d), np.diag([0]*N*d) ))
#     lambda_var = cp.Variable()
    lambda_var = cp.Variable(nonneg=True)


    gamma_shape = np.shape(d_supp)[0]
    gamma_matrix = []
    for i in range(N_sample):
        for j in range(N):
            gamma_var = cp.Variable((gamma_shape,1),nonneg = True)
#             gamma_var = cp.Variable([gamma_shape,1])
            gamma_matrix += [gamma_var]
    # k in N, i in N_sample
    #bk + <ak,xi_i>
    X_constraint = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde) @ W_sample_matrix_ext
#     si_var = cp.Variable((N_sample,1))
    si_var = cp.Variable(N_sample)

    for i in range(N_sample):
        for j in range(N):
#             print(N_sample)
            constraint_temp = X_constraint[n * (j+1) + i_th_state,i] + gamma_matrix[i * N + j].T @ (d_supp - C_supp @ W_sample_matrix[:, [i]])
#             constraint += [constraint_temp <= si_var[i,0]]
            constraint += [constraint_temp <= si_var[i]]


    ak_matrix = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde)[:,1:]
    for i in range(N_sample):
        for j in range(N):
#             constraint_temp = C_supp.T @ gamma_matrix[i * N + j] - ak_matrix[n * (j+1) + i_th_state:n * (j+1)+i_th_state + 1,:].T
            constraint_temp = C_supp.T @ gamma_matrix[i * N + j] - ak_matrix[[n * (j+1) + i_th_state],:].T
#             constraint += [cp.norm_inf(constraint_temp) <= lambda_var]
            constraint += [cp.norm(constraint_temp,p=np.inf) <= lambda_var]

#     for i in range(N_sample):
#         for j in range(N):
#             constraint += [gamma_matrix[i * N + j] >= 0]
#     constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= i_state_ub]
    constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= i_state_ub]
    return lambda_var, gamma_matrix, si_var, constraint

In [281]:
def define_obj(n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, beta = 0.95, N_sample = 6, i_th_state = 1, i_state_ub = 0.05, epsilon = 1, sin_const = 1):
    Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func = define_loss_func(m, d, r, Nu, Nw, Bx, Cx_tilde, mu, beta, sin_const)
    W_sample_matrix, W_sample_matrix_ext = gene_disturbance(N, d, N_sample, sin_const)
    lambda_var, gamma_matrix, si_var, constraint = define_constraint(W_sample_matrix, W_sample_matrix_ext, eigvec, H_cal_dec, H, H_new, n, Bx, Cx_tilde, Cy_tilde, Ey_tilde, N_sample, sin_const, i_th_state, i_state_ub, epsilon)
    return Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func, W_sample_matrix, W_sample_matrix_ext, lambda_var, gamma_matrix, si_var, constraint

In [282]:
def simulation(Ak, Bk, Ck, D, E, N,
               n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, delta_t, x_init, 
               i_state_ub = 0.05, beta = 0.95, i_th_state = 1, epsilon = 1, sin_const = 1, 
               N_sample = 6, N_sim = 80):

    
    
    t0 = 0
    xk = x_init
    uk = 0
    t = t0
    h = delta_t



    x_list = []
    x_list += xk.flatten().tolist()
    u_list = []

    for i in range(N_sim):
    #     if i % N == 0:
        n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak, Bk, Ck, D, E, xk, N)
        Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func, W_sample_matrix, W_sample_matrix_ext, lambda_var, gamma_matrix, si_var, constraint = define_obj(n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, 
        beta = beta, N_sample = N_sample, i_th_state = i_th_state, i_state_ub = i_state_ub, epsilon = epsilon, sin_const=sin_const)
        obj = cp.Minimize(loss_func)
        prob = cp.Problem(obj,constraint)
    #         print(W_sample_matrix)
        prob.solve(solver = cp.MOSEK)
#         print("opt value:", prob.value)
    #     print( prob.solve(verbose=True))
#         prob.solve(solver = cp.MOSEK,verbose = True, mosek_params = {mosek.dparam.basis_tol_s:1e-9, mosek.dparam.ana_sol_infeas_tol:0})
    #         print(Ax @ x_init +  Bx @ H.value @ W_sample_matrix_ext[:,0:1]  + Cx @ W_sample_matrix[:,0:1])
#         print("status:", prob.status)
#         print("Controller", H_cal_dec.value[0,0], H_cal_dec.value[0,1])
#         print("dual:", constraint[0].dual_value)
#         print("gamma", gamma_matrix[0].value,  gamma_matrix[1].value,  gamma_matrix[2].value,  gamma_matrix[3].value)
#         print("lambda",lambda_var.value)
#         print("lambda time epsilon",lambda_var.value * epsilon)
#         print("si",si_var.value)
#         print("si average",np.sum(si_var.value)/N_sample)
#         print("state_constraint", np.mean(si_var.value) + lambda_var.value * epsilon)
#         print("state",(Bx @ H_cal_dec.value @ (Cy_tilde + Ey_tilde) + Cx_tilde) @ W_sample_matrix_ext)
#         print("disturbance data", W_sample_matrix)
        wk = sin_const * np.sin(np.random.randn(d,1))
        uk = H_cal_dec.value[0,0] + H_cal_dec.value[0,1] * (D @ xk + E @ wk)
        u_list +=  uk.flatten().tolist() 
        print("current state and input", xk,uk)
#         print(wk)
#         x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
        x_kp1 = simulation_Euler(Ak, Bk, xk, uk)
        x_list += x_kp1.flatten().tolist()
        xk = x_kp1
#         xk += Ck @ wk
    return x_list

In [283]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

x_init = np.array([[-2],[0]])

In [284]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [285]:
Ak,Bk = disc_linear_system(A,B,delta_t)
Ck = np.array([[0, 0],[1e-2, 0]])
# Ck = np.array([[1e-2, 0],[0, 0]])
D = np.array([[1, 0]])
E = np.array([[0, 1e-2]])

N = 5
n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

Q = np.diag([10,1])
Qf = np.diag([15,1])
R = np.diag([1])

In [296]:
mu =np.zeros([d,1])
mu_w = np.vstack([1]+ [mu] * N)
M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)
beta = 0.95
N_sample = 1
i_th_state = 1 
i_state_ub = 0.4
epsilon = 10
sin_const = 3

N_sim = 30
N_loop = 5

sim_temp = simulation(Ak, Bk, Ck, D, E, N, n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, delta_t, x_init, 
               i_state_ub = i_state_ub, beta = beta, i_th_state = i_th_state, epsilon = epsilon, sin_const = sin_const, N_sample = N_sample, N_sim = N_sim)

current state and input [[-2]
 [ 0]] [[-1.73858676]]
current state and input [[-1.99025253]
 [ 0.09747471]] [[-2.07256171]]
current state and input [[-1.97362564]
 [ 0.16626886]] [[-2.35907819]]
current state and input [[-1.95244655]
 [ 0.21179096]] [[-2.52679389]]
current state and input [[-1.92824862]
 [ 0.24197924]] [[-2.61734057]]
current state and input [[-1.90204715]
 [ 0.26201472]] [[-2.69850041]]
current state and input [[-1.87469418]
 [ 0.27352967]] [[-2.71623584]]
current state and input [[-1.8466608 ]
 [ 0.28033387]] [[-2.69901141]]
current state and input [[-1.81820826]
 [ 0.28452533]] [[-2.66145955]]
current state and input [[-1.78947782]
 [ 0.28730447]] [[-2.61342598]]
current state and input [[-1.76054843]
 [ 0.28929392]] [[-2.55920335]]
current state and input [[-1.7314632 ]
 [ 0.29085224]] [[-2.50164107]]
current state and input [[-1.70224626]
 [ 0.2921694 ]] [[-2.44203285]]
current state and input [[-1.67291071]
 [ 0.2933555 ]] [[-2.38116245]]
current state and input 

In [337]:
w_sample = []
N_sample = 1
for i in range(N_sample * N):
    w_temp = sin_const * np.sin(np.random.randn(d,1))
    w_sample += [w_temp]

w_sample


[array([[-2.77995639],
        [-2.52645498]]),
 array([[ 2.83776228],
        [-1.67857256]])]